# Generating templates and recordings with non-rigid drifts

This notebook shows how to generate templates and recordings from scratch using MEArec.

The last part of the notebook showcases some plotting routines available through the package.

In [ ]:
import MEArec as mr
import MEAutility as mu
import yaml
from pprint import pprint
import matplotlib.pylab as plt
import numpy as np
from pathlib import Path

%matplotlib notebook

## Load default configuration files

First, let's load the default configuration of MEArec

In [ ]:
default_info, mearec_home = mr.get_default_config()
pprint(default_info)

## Generating and saving templates

In [ ]:
# define cell_models folder
cell_folder = default_info['cell_models_folder']
template_params = mr.get_default_templates_params()
pprint(template_params)

Now let's change a few parameters and generate templates. We need to generate templates with drift. In order to use the advanced drift features, we have to make sure that all templates drift roughly in the same direction and by the same distance.

In [ ]:
template_params["drifting"] = True
template_params["drift_steps"] = 30
# this ensures that all cells drift on the same z trajectory, with a small xy variation
template_params["drift_xlim"] = [-5, 5]
template_params["drift_ylim"] = [-5, 5]
template_params["drift_zlim"] = [100, 100]
template_params["max_drift"] = 200

# let's generate 50 templates per cell models (total 650 templates)
template_params["n"] = 20
template_params['probe'] = 'Neuronexus-32'

In [ ]:
templates_path = Path("data/test_drift_templates.h5")

In [ ]:
if not templates_path.is_file():
    # the templates are not saved, but the intracellular simulations are saved in 'templates_folder'
    tempgen = mr.gen_templates(cell_models_folder=cell_folder, params=template_params, n_jobs=13, verbose=1)
    # this will take a few minutes...
else:
    tempgen = mr.load_templates(templates_path, return_h5_objects=False)

The `tempgen` variable is a `TemplateGenerator` object. It contains the `templates`, `locations`, `rotations`, and `celltypes` of the generated templates.

In [ ]:
print('Templates shape', tempgen.templates.shape)

In [ ]:
# plot locations
probe = mu.return_mea(info=tempgen.info["electrodes"])

ax_probe = mu.plot_probe(probe)
for loc in tempgen.locations[::5]:
    ax_probe.plot([loc[0, 1], loc[-1, 1]], [loc[0, 2], loc[-1, 2]], alpha=0.7)

We can now save the `TemplateGenerator` object in h5 format.

In [ ]:
if not templates_path.is_file():
    # save templates in h5 format
    mr.save_template_generator(tempgen, filename=templates_path)

## Generating and saving drift recordings with 1 drift signal

Once the templates have been generated, we can use them to generate recordings. Let's fisrt load and take a look at the default parameters:

In [ ]:
recordings_params = mr.get_default_recordings_params()
pprint(recordings_params)

The default drift parameters can be used to generate a **single drift signal**. We'll see at the end of the notebook how to generate more complex drifting modes.

Similarly to the templates generation, we can change th eparameters that we pass to the `gen_recordings` function.
In this case we will keep the default parameters.

In [ ]:
# 10 min
recordings_params["spiketrains"]["duration"] = 600

# 100 Excitatory, 20 inhibitory (the main difference is morphology and avg firing rates)
recordings_params["spiketrains"]["n_exc"] = 15
recordings_params["spiketrains"]["n_inh"] = 5

# set template selection params
recordings_params["templates"]["min_amp"] = 30
recordings_params["templates"]["min_dist"] = 20 # um 

# other settings
recordings_params["recordings"]["filter"] = True

# noise level and model
recordings_params["recordings"]["noise_level"] = 10
recordings_params["recordings"]["noise_mode"] = "distance-correlated"

# set chunk duration (IMPORTANT for RAM usage and parallelization)
recordings_params["recordings"]["chunk_duration"] = 10

# drifting option
recordings_params["recordings"]["drifting"] = True
recordings_params["recordings"]["slow_drift_velocity"] = 30
recordings_params["recordings"]["slow_drift_amplitude"] = 30
recordings_params["recordings"]["t_start_drift"] = 100
recordings_params["recordings"]["t_end_drift"] = 500

recordings_params["recordings"]["drift_mode_probe"] = 'rigid'


recordings_params["recordings"]["filter"] = False


# (optional) set seeds for reproducibility 
# (e.g. if you want to maintain underlying activity, but change e.g. noise level)
recordings_params['seeds']['spiketrains'] = None
recordings_params['seeds']['templates'] = None
recordings_params['seeds']['convolution'] = None
recordings_params['seeds']['noise'] = None



In [ ]:
recgen_rigid = mr.gen_recordings(templates='data/test_drift_templates.h5', 
                                 params=recordings_params, verbose=True, 
                                 n_jobs=10)

In [ ]:
mr.plot_cell_drifts(recgen_rigid)

In [ ]:
recordings_params["recordings"]["drift_mode_probe"] = 'non-rigid'
recordings_params["recordings"]["non_rigid_gradient_mode"] = 'linear'
recordings_params["recordings"]["non_rigid_linear_direction"] = 1

In [ ]:
recgen_non_rigid = mr.gen_recordings(templates='data/test_drift_templates.h5', 
                                     params=recordings_params, verbose=True,
                                     n_jobs=10)

In [ ]:
plt.figure()
plt.plot(recgen_rigid.drift_list[0]['drift_vector_idxs'])

In [ ]:
mr.plot_cell_drifts(recgen_non_rigid)

In [ ]:
recordings_params["recordings"]["drift_mode_probe"] = 'non-rigid'
recordings_params["recordings"]["non_rigid_gradient_mode"] = 'linear'
recordings_params["recordings"]["non_rigid_linear_direction"] = -1

In [ ]:
recgen_non_rigid_rev = mr.gen_recordings(templates='data/test_drift_templates.h5', 
                                         params=recordings_params, verbose=True,
                                         n_jobs=10)

In [ ]:
mr.plot_cell_drifts(recgen_non_rigid_rev)

In [ ]:
recordings_params["recordings"]["drift_mode_probe"] = 'non-rigid'
recordings_params["recordings"]["non_rigid_gradient_mode"] = 'step'
recordings_params["recordings"]["non_rigid_step_factors"] = (1, 0.1)

In [ ]:
recgen_non_rigid_step = mr.gen_recordings(templates='data/test_drift_templates.h5', 
                                          params=recordings_params, verbose=True,
                                          n_jobs=10)

In [ ]:
mr.plot_cell_drifts(recgen_non_rigid_step)

In [ ]:
# save recordings in h5 format
mr.save_recording_generator(recgen_rigid, filename='data/test_drift_recordings_rigid.h5')
mr.save_recording_generator(recgen_non_rigid, filename='data/test_drift_recordings_nonrigid.h5')
mr.save_recording_generator(recgen_non_rigid, filename='data/test_drift_recordings_nonrigid_rev.h5')
mr.save_recording_generator(recgen_non_rigid, filename='data/test_drift_recordings_nonrigid_step.h5')

In [ ]:
recgen_rigid_loaded = mr.load_recordings('data/test_drift_recordings_rigid.h5')
recgen_non_rigid_loaded = mr.load_recordings('data/test_drift_recordings_nonrigid.h5')

## Use multiple drift signals

In order to construct more complicated drifts, we can use a combination of multiple drift signals.
In this example, we show how to combine 3 drift signals:
1. a slow rigid drift with velocity 10um/min
2. a fast rigi drift with events every 60s
3. an additional non-rigid fast sine oscillation

In [ ]:
drift_dict_slow = mr.get_default_drift_dict()
drift_dict_fast = mr.get_default_drift_dict()
drift_dict_slow_nonrigid = mr.get_default_drift_dict()

In [ ]:
drift_dict_slow["drift_mode_probe"] = "rigid"
drift_dict_slow["drift_fs"] = 5
drift_dict_slow["slow_drift_velocity"] = 10
drift_dict_slow["slow_drift_amplitude"] = 30

In [ ]:
drift_dict_fast["drift_mode_speed"] = "fast"
drift_dict_fast["fast_drift_period"] = 60
drift_dict_fast["fast_drift_max_jump"] = 15

In [ ]:
drift_dict_slow_nonrigid["drift_mode_probe"] = "non-rigid"
drift_dict_slow_nonrigid["drift_mode_speed"] = "slow"
drift_dict_slow_nonrigid["slow_drift_waveform"] = "sine"
drift_dict_slow_nonrigid["slow_drift_velocity"] = 80
drift_dict_slow_nonrigid["slow_drift_amplitude"] = 10

In [ ]:
drift_dicts = [drift_dict_slow, drift_dict_fast, drift_dict_slow_nonrigid]

In [ ]:
recgen_mixed = mr.gen_recordings(templates='data/test_drift_templates.h5', 
                                 params=recordings_params, verbose=True,
                                 n_jobs=1, drift_dicts=drift_dicts)

In [ ]:
mr.plot_cell_drifts(recgen_mixed)

In [ ]:
mr.save_recording_generator(recgen_mixed, filename='data/test_drift_recordings_mixed.h5')
recgen_mixed_loaded = mr.load_recordings('data/test_drift_recordings_mixed.h5')

In [ ]:
recgen_mixed.template_ids

### Use an external drift signal

In this section we combine a slow drift with an externally-defined fast drift with an exponential decay shape.
The fast drift is only transient and is applied to neurons only in a section of the probes. To do that, we use `template_ids` to pre-select templates and we define `external_drift_vectors` based on their locations.

In [ ]:
drift_dict_external = mr.get_default_drift_dict()

In [ ]:
# we define a time vector from 100 tp 150 seconds, with a 0.1s period
drift_times = np.arange(100, 150, 0.1)

In [ ]:
tau = 5
amplitude = 30
drift_vector_um = np.zeros_like(drift_times)
# let's leave the first 50 samples as zeros
start_index = 50
drift_vector_um[start_index:] = amplitude * np.exp(-(drift_times[:-start_index] - drift_times[0]) / tau)

In [ ]:
plt.figure()
plt.plot(drift_times, drift_vector_um)

Now we compute drift vectors. We will apply this drift only to cells between 0-50 um in depth

In [ ]:
template_ids = recgen_mixed.template_ids
template_locations = tempgen.locations[template_ids]

In [ ]:
template_locations.shape

In [ ]:
drift_steps = template_locations.shape[1]
# start position is half the drift range
drift_direction = 2 # depth
drift_factors = np.zeros(len(template_ids))
mask = np.logical_and(template_locations[:, drift_steps // 2, drift_direction] <= 50,
                      template_locations[:, drift_steps // 2, drift_direction] >= 0) 
drift_factors[mask] = 1

In [ ]:
drift_dict_external["external_drift_times"] = drift_times
drift_dict_external["external_drift_vector_um"] = drift_vector_um
drift_dict_external["external_drift_factors"] = drift_factors

In [ ]:
drift_dicts = [drift_dict_slow, drift_dict_external]

In [ ]:
recgen_mixed_ext = mr.gen_recordings(templates='data/test_drift_templates.h5', 
                                     params=recordings_params, verbose=True,
                                     n_jobs=10, drift_dicts=drift_dicts)

In [ ]:
mr.plot_cell_drifts(recgen_mixed_ext)